In [3]:
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
print(azureml.core.VERSION)

subscription_id="your id
resource_group="your resource grpop"
workspace_name="your workspace"
compute="your compute name"

# Load the workspace from the saved config file
ws = Workspace.from_config()


#Create New Experiment
exp = Experiment(ws, name = "BasicNLP")

import datetime
run = exp.start_logging(snapshot_directory=None)
run.log("Experimentation Start Time",str(datetime.datetime.now()))




1.51.0


In [6]:

!pip install kaggle

  Using cached kaggle-1.6.6.tar.gz (84 kB)
  Using cached python_slugify-8.0.4-py2.py3-none-any.whl (10 kB)
  Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)
  Created wheel for kaggle: filename=kaggle-1.6.6-py3-none-any.whl size=111946 sha256=8b2807284ae8c6c79f3855e43723c0f23001ca6db7f96ac9c9e26f1561391718
  Stored in directory: /home/azureuser/.cache/pip/wheels/0a/e0/6d/fbc9dff0a755904e8a1a5e0730d0918114270dc17c391c9a91
Successfully built kaggle


In [4]:
import os
from azureml.core import Workspace, Dataset

# Load AzureML workspace
ws = Workspace.from_config()

# Authenticate Kaggle (if necessary)
# You can authenticate Kaggle using your Kaggle API token or username and API key

# Set Kaggle configuration (replace 'username' and 'api_key' with your actual Kaggle credentials)
#os.environ['KAGGLE_USERNAME'] = 'your username'
#os.environ['KAGGLE_KEY'] = 'your key

# Download Kaggle dataset
#!kaggle competitions download -c nlp-getting-started

# Unzip downloaded dataset (if necessary)
#!unzip nlp-getting-started.zip

# Upload dataset to AzureML datastore
ds = ws.get_default_datastore()
#ds.upload(src_dir='./', target_path='DisasterTweet', overwrite=True, show_progress=True)

# Create dataset from the uploaded files
dataset = Dataset.File.from_files(path=(ds, 'DisasterTweet/*.csv'))

# Register dataset
#dataset = dataset.register(workspace=ws, name='DisasterTweet', description='DisasterTweet_dataset')


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [6]:
#train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
#test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
default_ds = ws.get_default_datastore()
#dataset = Dataset.get_by_name(ws, name='DisasterTweet')
train_dataset = Dataset.Tabular.from_delimited_files(path=(default_ds, 'DisasterTweet/train.csv'))
train_df = train_dataset.to_pandas_dataframe()
test_dataset = Dataset.Tabular.from_delimited_files(path=(default_ds, 'DisasterTweet/test.csv'))
test_df = test_dataset.to_pandas_dataframe()
sub_dataset = Dataset.Tabular.from_delimited_files(path=(default_ds, 'DisasterTweet/sample_submission.csv'))
sub_df = sub_dataset.to_pandas_dataframe()
train_df.head()

,id,keyword,location,text,target
0,1,None,None,Our Deeds are the Reason of this #earthquake M...,1.0
1,4,None,None,Forest fire near La Ronge Sask. Canada,1.0
2,5,None,None,All residents asked to 'shelter in place' are ...,1.0
3,6,None,None,"13,000 people receive #wildfires evacuation or...",1.0
4,7,None,None,Just got sent this photo from Ruby #Alaska as ...,1.0


In [7]:
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

In [8]:
train_df[train_df["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

# Building vectors
The theory behind the model is the words contained in each tweet are a good indicator of whether they're about a real disaster or not.

We'll use scikit-learn's CountVectorizer to count the words in each tweet and turn them into data our machine learning model can process.

Note: a vector is, in this context, a set of numbers that a machine learning model can work with. 

In [9]:
count_vectorizer = feature_extraction.text.CountVectorizer()

## let's get counts for the first 5 tweets in the data
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])


In [10]:
## we use .todense() here because these vectors are "sparse" (only non-zero elements are kept to save space)
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())


(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


when running for the entire dataset, its showing none object type, so we have to check for missing values

In [ ]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = count_vectorizer.transform(test_df["text"])

In [11]:
# Check for missing values in the entire DataFrame
missing_values = train_df.isnull().sum()

# Display the count of missing values for each column
print(missing_values)


id             0
keyword      404
location    3311
text         782
target      1217
dtype: int64


In [12]:
# Check for missing values in the "text" column
missing_values_text = train_df["text"].isnull().sum()

# Display the count of missing values in the "text" column
print("Missing values in 'text' column:", missing_values_text)


Missing values in 'text' column: 782


# Use a pre-trained language model to generate text and fill missing values

# #Alternative Method:
#Train a text generation model on the existing data and generate text to fill in missing values

In [ ]:
!pip install --upgrade tensorflow

In [33]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

# Assuming train_df['text'] contains your text data with missing values

# Step 1: Prepare the Data
train_df['text'].fillna("", inplace=True)  # Replace missing values with an empty string

# Step 2: Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['text'])

# Step 3: Convert text to sequences
sequences = tokenizer.texts_to_sequences(train_df['text'])

# Step 4: Padding sequences
max_len = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

# Step 5: Model Training
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_len))
model.add(LSTM(100))
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Assuming you have target values for supervised learning
# target_values = train_df['target'].values
# Convert target values to one-hot encoding if needed

# model.fit(padded_sequences, target_values, epochs=10, validation_split=0.2)

# Step 6: Generate Text for Missing Values
missing_indices = train_df[train_df['text'] == ""].index
for idx in missing_indices:
    sequence = tokenizer.texts_to_sequences([train_df.at[idx, 'text']])
    padded_sequence = pad_sequences(sequence, maxlen=max_len, padding='post')
    predicted_sequence = model.predict(padded_sequence)
    #predicted_text = tokenizer.sequences_to_texts([np.argmax(predicted_sequence)])[0]
    # Convert numpy array to list
    predicted_sequence_list = predicted_sequence[0].tolist()
    # Convert sequence to text
    predicted_text = tokenizer.sequences_to_texts([predicted_sequence_list])[0]


    train_df.at[idx, 'text'] = predicted_text

# Step 7: Imputation
# The missing values in the 'text' column have been filled with generated text


2.11.0
1/1 [==============================] - 0s 21ms/step


In [36]:
# Check for missing values in the "text" column
missing_values_text = test_df["text"].isnull().sum()

# Display the count of missing values in the "text" column
print("Missing values in 'text' column:", missing_values_text)

Missing values in 'text' column: 354


In [37]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

# Assuming test_df['text'] contains your text data with missing values

# Step 1: Prepare the Data
test_df['text'].fillna("", inplace=True)  # Replace missing values with an empty string

# Step 2: Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(test_df['text'])

# Step 3: Convert text to sequences
sequences = tokenizer.texts_to_sequences(test_df['text'])

# Step 4: Padding sequences
max_len = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

# Step 5: Model Training
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_len))
model.add(LSTM(100))
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Assuming you have target values for supervised learning
# target_values = train_df['target'].values
# Convert target values to one-hot encoding if needed

# model.fit(padded_sequences, target_values, epochs=10, validation_split=0.2)

# Step 6: Generate Text for Missing Values
missing_indices = test_df[test_df['text'] == ""].index
for idx in missing_indices:
    sequence = tokenizer.texts_to_sequences([test_df.at[idx, 'text']])
    padded_sequence = pad_sequences(sequence, maxlen=max_len, padding='post')
    predicted_sequence = model.predict(padded_sequence)
    #predicted_text = tokenizer.sequences_to_texts([np.argmax(predicted_sequence)])[0]
    # Convert numpy array to list
    predicted_sequence_list = predicted_sequence[0].tolist()
    # Convert sequence to text
    predicted_text = tokenizer.sequences_to_texts([predicted_sequence_list])[0]


    test_df.at[idx, 'text'] = predicted_text

# Step 7: Imputation
# The missing values in the 'text' column have been filled with generated text


2.11.0
1/1 [==============================] - 0s 25ms/step


In [38]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = count_vectorizer.transform(test_df["text"])


In [39]:
## Our vectors are really big, so we want to push our model's weights
## toward 0 without completely discounting different words - ridge regression 
## is a good way to do this.
clf = linear_model.RidgeClassifier()

In [42]:
# Check for missing values in the "text" column
missing_values_text = train_df["target"].isnull().sum()

# Display the count of missing values in the "text" column
print("Missing values in 'text' column:", missing_values_text)

Missing values in 'text' column: 1217


In [44]:
import numpy as np

# Replace missing values (NaNs) in the "target" column with an empty string
train_df['target'].fillna("", inplace=True)

# Filter out any non-string values and convert them to strings
train_df['target'] = train_df['target'].apply(lambda x: str(x) if isinstance(x, str) else "")

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['target'])


In [46]:
# Check for missing values in the "text" column
missing_values_text = train_df["target"].isnull().sum()

# Display the count of missing values in the "text" column
print("Missing values in 'text' column:", missing_values_text)

Missing values in 'text' column: 0


In [45]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/numpy/lib/arraysetops.py:604: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  >>> test = np.array([0, 1, 2, 5, 0])
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/numpy/lib/arraysetops.py:604: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  >>> test = np.array([0, 1, 2, 5, 0])
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` par

array([0., 0., 0.])

In [47]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier()

In [60]:
# Assuming clf.predict(test_vectors) returns predictions for all rows in test_vectors

# Check the length of predictions
print(len(clf.predict(test_vectors)))

# Check the length of sub_df
print(len(sub_df))

# Make sure the lengths match

# Ensure proper alignment of DataFrame index and predictions
#sub_df["target"] = clf.predict(test_vectors)[:len(sub_df)].tolist()
# Ensure proper alignment of DataFrame index and predictions
sub_df["target"] = clf.predict(test_vectors)[:len(sub_df)].tolist()



3615
3263


In [64]:
import joblib
# Save the trained model in the outputs folder
import os
print("Saving model...")
os.makedirs('outputs', exist_ok=True)
model_file = os.path.join('outputs', 'DisasterTweets_model.pkl')
joblib.dump(value=clf, filename=model_file)


# Register the model
print('Registering model...')
Model.register(workspace=run.experiment.workspace,
               model_path = model_file,
               model_name = 'DisasterTweet_model')

Saving model...
Registering model...
Registering model DisasterTweet_model


Model(workspace=Workspace.create(name='ml-cranium-ws', subscription_id='6b2588af-0daa-428d-9456-c07e5d3297e6', resource_group='ml-cranium-poc-rg'), name=DisasterTweet_model, id=DisasterTweet_model:1, version=1, tags={}, properties={})

In [65]:
import tempfile
import os
# Fetches latest model
model = ws.models['DisasterTweet_model']
print(model.name, 'version', model.version)

# Download the model file to a temporary directory
temp_dir = tempfile.mkdtemp()
model.download(target_dir=temp_dir, exist_ok=True)

# Get the path to the downloaded model file
model_path = os.path.join(temp_dir, 'DisasterTweets_model.pkl')
loaded_model = joblib.load(model_path)
#y = loaded_model.predict([[0,148,72,35,0,33.6,0.627, 50]])
sub_df["target"] = clf.predict(test_vectors)[:len(sub_df)].tolist()
print("the Output is:", sub_df["target"])

DisasterTweet_model version 1
the Output is: 0        
1        
2        
3        
4        
       ..
3258     
3259     
3260     
3261     
3262     
Name: target, Length: 3263, dtype: object


In [66]:
run.log("experiment End Time:", str(datetime.datetime.now()))
run.complete()

In [67]:
print(run.get_portal_url)

<bound method HasRunPortal.get_portal_url of Run(Experiment: BasicNLP,
Id: 63606d77-ab5a-419d-901a-5bae1bcc4b91,
Type: None,
Status: NotStarted)>


In [57]:
#sub_dataset = Dataset.Tabular.from_delimited_files(path=(default_ds, 'DisasterTweet/sample_submission.csv'))
#sub_df = sub_dataset.to_pandas_dataframe()
#sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
sub_df["target"] = clf.predict(test_vectors)


ValueError: Length of values (3615) does not match length of index (3263)

In [63]:
sub_df["target"].head()

0    
1    
2    
3    
4    
Name: target, dtype: object

In [62]:
sub_df.to_csv("submission.csv", index=False)